In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
import mysql.connector
import ssl

# konksi nanti coba ditaruh diluar
# koneksi
db = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='UnNgGrape'
    )
cursor = db.cursor()

class Scrape:

    def scrapeMain(keyword):
        # inisialisasi array untuk menampung hasil data
        asal_situs = []
        title_lowongan = []
        nama_perusahaan = []
        lokasi_perusahaan = []
        keterangan_lowongan = []
        skill_lowongan = []
        benefit_lowongan = []
        deskripsi_lowongan = []
        stem_detail = []
        link_lowongan = []

        # pake try except (continue) kalau udah looping keyword
        # loop scrape
        cursor.execute("SELECT * FROM `scrape` WHERE `asal_situs` LIKE 'jobstreet'")
        raw_scrape = cursor.fetchall()
        for rowS in raw_scrape:
            asal = rowS[0]
            # scrape data
            main_link1 = rowS[1]
            main_link2 = rowS[2]
            main_link = main_link1 + keyword + main_link2
            tag_main = rowS[3]
            tag_lowongan = rowS[4]
            tag_lowongan_part = rowS[5]
            tag_perusahaan = rowS[6]
            tag_perusahaan_part = rowS[7]
            tag_lokasi = rowS[8]
            tag_lokasi_part = rowS[9]
            tag_mainDetail = rowS[10]
            tag_keterangan = rowS[11]
            tag_skill = rowS[12]
            tag_benefit = rowS[13]
            tag_deskripsi = rowS[14]
            breakDeskripsi = rowS[15]
            raw_link = rowS[16]

            # tab sekali jika sudah masuk looping scrape
            r = Request(main_link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
            response = urlopen(r).read()
            soup = BeautifulSoup(response, "lxml")
            print(type(soup))

            jobList = soup.find_all("div", tag_main)
            for p in jobList:
                # link scrape dulu untuk cek data sudah pernah di scrape atau belum
                link = raw_link+p.find('a').get('href')
                lowongan = p.find(tag_lowongan_part, tag_lowongan).get_text()
                # menggunakan try except karena ada beberapa perusahaan yang dirahasiakan
                try:
                    perusahaan = p.find(tag_perusahaan_part, tag_perusahaan).get_text()
                except:
                    perusahaan = "Perusahaan Dirahasiakan"

                try:
                    lokasi = p.find(tag_lokasi_part, tag_lokasi).get_text().replace("head office - ", "")
                except:
                    lokasi = "-"

                # try print return scrapeDetail
                keterangan, skill, benefit, deskripsi = Scrape.scrapeDetail(link, tag_mainDetail, tag_keterangan, tag_skill, tag_benefit, tag_deskripsi, breakDeskripsi)

                print(Scrape.scrapeDetail(link, tag_mainDetail, tag_keterangan, tag_skill, tag_benefit, tag_deskripsi, breakDeskripsi))
                print("\n")

                # stemming for detail
                raw_stem = lowongan + " " + perusahaan + " " + lokasi + " " + str(keterangan) + " " + str(skill) + " " + str(benefit) + " " + str(deskripsi)
                stem = Scrape.stemming(raw_stem)

                # push to array penampung hasil data
                asal_situs.append(asal)
                title_lowongan.append(lowongan)
                nama_perusahaan.append(perusahaan)
                lokasi_perusahaan.append(lokasi)
                keterangan_lowongan.append(keterangan)
                skill_lowongan.append(skill)
                benefit_lowongan.append(benefit)
                deskripsi_lowongan.append(deskripsi)
                stem_detail.append(stem)
                link_lowongan.append(link)

        # tampil pakai panda
        jobList_dict ={'asal':asal_situs, 'lowongan':title_lowongan, 'perusahaan':nama_perusahaan, 'lokasi':lokasi_perusahaan, 'keterangan':keterangan_lowongan, 'skill':skill_lowongan, 'benefit':benefit_lowongan, 'deskripsi':deskripsi_lowongan, 'stem':stem_detail, 'link':link_lowongan}
        df = pd.DataFrame(jobList_dict,columns = ['asal', 'lowongan', 'perusahaan', 'lokasi', 'keterangan', 'skill', 'benefit', 'deskripsi', 'stem', 'link'])
        return df.sort_values('asal',ascending=True)

    def scrapeDetail(linkDetail, tag_mainDetail, tag_keterangan, tag_skill, tag_benefit, tag_deskripsi, breakDeskripsi):
        r = Request(linkDetail, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
        response = urlopen(r).read()
        soup = BeautifulSoup(response, "lxml")

        raw_detail = soup.find_all("div", tag_mainDetail)
        for p in raw_detail:
            # menggunakan try except karena ada beberapa estimasi keterangan yang tidak terlampir
            try:
                raw_keterangan = p.find("div", tag_keterangan ).get_text(separator=". ").replace("Fungsi Kerja. ", "")
                nltk_tokens = nltk.sent_tokenize(raw_keterangan)
                keterangan = ""

                for x in nltk_tokens:
                    if x == "Lamar.":
                        break
                    keterangan = keterangan + " " + x
            except:
                keterangan = "-"
            
            try:
                skill = p.find("div", tag_skill).get_text(separator=" ")
            except:
                skill = "-"
            try:
                benefit = p.find("div", tag_benefit).get_text(separator=" ").replace("Tunjangan dan keuntungan", "keuntungan:")
            except:
                benefit = "-"

            # menggunakan try except karena ada beberapa deskripsi yang NoneType
            try:
                # replace untuk glints
                raw_deskripsi = p.find("div", tag_deskripsi ).get_text(separator=". ").replace("Informasi Penting. Pastikan perusahaan yang kamu lamar resmi dengan memeriksa website dan lowongan kerja mereka.. Read Less.", "")
                nltk_tokens = nltk.sent_tokenize(raw_deskripsi)
                deskripsi = ""

                for x in nltk_tokens:
                    if x == breakDeskripsi:
                        break
                    deskripsi = deskripsi + " " + x
            except:
                deskripsi = "-"
            
            # give to scrapeMain
            return keterangan, skill, benefit, deskripsi

    def stemming(raw_stem):
        # define punctuation
        punctuations = '''!()-[]{};:=+`'",<>./|\?@#$%^&*_~'''

        # remove punctuation from the string
        no_punct = ""
        for char in raw_stem:
            if char not in punctuations:
                no_punct = no_punct + char
            else:
                no_punct = no_punct + " "

        # display the unpunctuated string
        raw_stem = no_punct.lower()

        # create stemmer
        # nltk
        ps = PorterStemmer()
        # sastrawi
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()

        # stemming process
        # nltk
        words = word_tokenize(raw_stem)
        nltk_stemmer = ""
        for w in words:
            nltk_stemmer = nltk_stemmer + " " + ps.stem(w)
        # sastrawi sekaligus return
        return stemmer.stem(nltk_stemmer)

# print(Scrape.getKeyword())
# print("\n")
# Scrape.scrapeMain() 
Scrape.scrapeMain("palembang")

<class 'bs4.BeautifulSoup'>
('-', '-', '-', ' Deskripsi Pekerjaan. Deskripsi Pekerjaan :. Membantu Kepala Cabang untuk merumuskan, melaksanakan kebijakan atau strategi cabang di bidang teknik yaitu mencapai hasil underwriting sesuai dengan target yang telah ditetapkan.. Menyusun strategi operasional di cabang untuk mencapai target yang ditetapkan.. Melakukan analisa resiko dan survey.. Membuat laporan-laporan secara akurat dan tepat.. Meningkatkan kualitas SDM di bidang asuransi cabang melalui pendidikan dan pelatihan.. Memberikan masukan kepada Kepala Cabang atas perkembangan di cabang.. Kualifikasi :. Memiliki Sertifikasi Gelar Profesi AAAIK/AAIK. Minimal Pendidikan S1. Mampu mengoperasikan komputer (khususnya program : word processor, spread sheet,presentation aplikasi care, ms office dan email/internet). Informasi Tambahan. Tingkat Pekerjaan. Manajer/Asisten Manajer. Kualifikasi. Tidak terspesifikasi. Pengalaman Kerja. 3 tahun. Jenis Pekerjaan. Penuh Waktu. Spesialisasi Pekerjaan. 

,asal,lowongan,perusahaan,lokasi,keterangan,skill,benefit,deskripsi,stem,link
0,jobstreet,Manager Underwriting,PT Asuransi Umum Bumiputera Muda 1967,"Palembang, Bangka Belitung, Manado, Pekanbaru,...",-,-,-,Deskripsi Pekerjaan. Deskripsi Pekerjaan :. M...,manag underwrit pt asuransi umum bumiputera mu...,https://www.jobstreet.co.id/id/job/manager-und...
1,jobstreet,Branch Manager,PT Delisari Nusantara,"Malang, Banjarmasin, Pekanbaru, Makassar, Pale...",-,-,-,Keuntungan. Medical Insurance. BPJS Kesehatan...,branch manag pt sar nusantara malang banjarmas...,https://www.jobstreet.co.id/id/job/branch-mana...
2,jobstreet,Supervisor Purchasing,PT Priamanaya Energi,Palembang,-,-,-,Deskripsi Pekerjaan. Kualifikasi:. Minimal pe...,supervisor purchas pt priamanaya energi palemb...,https://www.jobstreet.co.id/id/job/supervisor-...
3,jobstreet,Administrasi Support (Indonesia Barat),Perusahaan Dirahasiakan,Perusahaan Dirahasiakan,-,-,-,Keuntungan. Budaya kerja dinamis. Kesempatan ...,administrasi support indonesia barat usaha rah...,https://www.jobstreet.co.id/id/job/administras...
4,jobstreet,Store Manager area Bandung,PT Pendekar Bodoh,Palembang,-,-,-,Deskripsi Pekerjaan. Gambaran Pekerjaan :. Me...,store manag area bandung pt pendekar bodoh pal...,https://www.jobstreet.co.id/id/job/store-manag...
5,jobstreet,Business Representative Area Palembang,PT Setia Anugrah Medika,"Palembang, Lampung",-,-,-,Keuntungan. Kesempatan berkarir yang cukup be...,busi repres area palembang pt setia anugrah me...,https://www.jobstreet.co.id/id/job/business-re...
6,jobstreet,Sales Representative (Medical),PT. Budimanmaju Megah Farmasi,"Jakarta Raya, Bogor, Bandung, Batam, Medan, Pa...",-,-,-,Deskripsi Pekerjaan. Kualifikasi:. Minimal 2 ...,sale repres medic pt budimanmaju megah farmasi...,https://www.jobstreet.co.id/id/job/sales-repre...
7,jobstreet,Area Sales Manager ( Sumatera Selatan ),PT URC Indonesia,Palembang,-,-,-,Deskripsi Pekerjaan. Business & Strategic. 1....,area sale manag sumatera selatan pt urc indone...,https://www.jobstreet.co.id/id/job/area-sales-...
8,jobstreet,SPG / SPB,PT Propan Raya ICC,"Palembang, Lampung, Pekanbaru, Medan",-,-,-,Deskripsi Pekerjaan. Syarat :. Usia maksimal ...,spg spb pt prop raya icc palembang lampung pek...,https://www.jobstreet.co.id/id/job/spg-spb-369...
9,jobstreet,Accounting Section Head,PT Sri Trang Lingga Indonesia,Palembang,-,-,-,Deskripsi Pekerjaan. General Ledger & Account...,account section head pt sri trang lingga indon...,https://www.jobstreet.co.id/id/job/accounting-...
